# Credmark Modeling Framework Example for Jupyter notebook
## Account - Tokens

version: 2023.4.15

## Initialize

<div class="alert alert-block alert-info">
    <b>Note:</b> Change to a web3 provider you have in `params` below.
</div>

In [13]:
%reload_ext credmark.cmf.ipython

cmf_param = {
    'chain_id': 1,
    'block_number': None,
    # 'chain_to_provider_url': {'1': 'https://mainnet.infura.io/v3/... or https://eth-mainnet.g.alchemy.com/'}, # or, use the credmark's nodes
    'api_url': None,
    'use_local_models': None,
    'register_utility_global': True}

context, model_loader = %cmf cmf_param

## Convex Token

In [14]:
convex_token = Token(address='0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B')

with context.ledger.TokenTransfer as q:
    df_convex = (q.select(
        columns=[q.BLOCK_NUMBER,
                 q.LOG_INDEX,
                 q.TO_ADDRESS,
                q.FROM_ADDRESS,
                q.TOKEN_ADDRESS],
        where=q.TOKEN_ADDRESS.eq(convex_token.address),
).to_dataframe().sort_values(['block_number', 'log_index']))
df_convex

,block_number,log_index,to_address,from_address,token_address
656,12452140,276,0x16f2f6eb3afd95ce06ed9b700ec7d44faf380c8a,0x2e088a0a19dda628b4304301d1ea70b114e4accd,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
1671,12452228,208,0xd7e228b7eb0c727706d5955c4c518cb10a3e7a19,0x2e088a0a19dda628b4304301d1ea70b114e4accd,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
3577,12452254,338,0x5e5ffef1103717f17a9a155d0c7646ec5e16551e,0x2e088a0a19dda628b4304301d1ea70b114e4accd,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
547,12452380,314,0x08e5e4df2f56be734ead5c80a3b2377616210a62,0x05767d9ef41dc40689678ffca0608878fb3de906,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
3331,12452434,153,0xf5eb2293bb18199fdf58f4f85fc9d4fbfea0638b,0x94a552a480642522f24f0fadba0b0dfab95708dc,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
...,...,...,...,...,...
706,17064523,166,0xa4c507a1eb3cd1bf2c1389f2d6cb93c9be971911,0x28c6c06298d514db089934071355e5743bf21d60,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
127,17064525,179,0x98c3d3183c4b8a650614ad179a1a98be0a8d6b8e,0x575e96f61656b275ca1e0a67d9b68387abc1d09c,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
702,17064535,3,0x9507c04b10486547584c37bcbd931b2a4fee9a41,0xb576491f1e6e5e62f1d8f26062ee822b40b0e0d4,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
112,17064555,149,0xa4c507a1eb3cd1bf2c1389f2d6cb93c9be971911,0x21a31ee1afc51d94c2efccaa2092ad1028285549,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b


## Account

In [15]:
acc_1 = Account(address=Address('0x5291fBB0ee9F51225f0928Ff6a83108c86327636'))

### Tokens that have been owned by the account

In [16]:
with context.ledger.TokenBalance as q:
    df_balance = q.select(
    where=q.ADDRESS.eq(acc_1.address),
    group_by=[q.TOKEN_ADDRESS]
    ).to_dataframe()

df_balance

,token_address
0,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599
1,0xdac17f958d2ee523a2206206994597c13d831ec7
2,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d
3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
4,0xd533a949740bb3306d119cc777fa900ba034cd52
5,0xc4ad29ba4b3c580e6d59105fff484999997675ff
6,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b


### Token Transfers for this account

In [17]:
with context.ledger.TokenTransfer as q:
       group_cols = [q.BLOCK_NUMBER,
                     q.LOG_INDEX,
                     q.TO_ADDRESS,
                     q.FROM_ADDRESS,
                     q.TRANSACTION_HASH,
                     q.TOKEN_ADDRESS]

       df_tt = (q.select(
                     aggregates=[
                            (f'sum(case when {q.TO_ADDRESS.eq(acc_1.address)} THEN {q.VALUE} else -{q.VALUE} END)',
                            'sum_value')],
                     where=(q.TO_ADDRESS.eq(acc_1.address).or_(q.FROM_ADDRESS.eq(acc_1.address))),
                     group_by=group_cols
                     )
                 .to_dataframe()
              .sort_values(['block_number', 'log_index']).reset_index())

df_tt

,index,block_number,log_index,to_address,from_address,transaction_hash,token_address,sum_value
0,37,14407839,58,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0xa9b26c2ba2e3a3176134be88bf1f95d7c7304aed,0x8f9d945275587b994caaf36656996f2012c6cf07b50f...,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,1068317764
1,20,14634606,220,0x93054188d876f558f4a66b2ef1d97d16edf0895b,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x2fbacf618cf121a2d74da84f6a1bc2697b031da48354...,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,-750000000
2,24,14634606,222,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x93054188d876f558f4a66b2ef1d97d16edf0895b,0x2fbacf618cf121a2d74da84f6a1bc2697b031da48354...,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,748139023
3,32,14649241,364,0x3993d34e7e99abf6b6f367309975d1360222d446,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0xf97d756f37c333657f425505a61d9e3c12329a6b69aa...,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,-748139023
4,8,14649241,369,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x3993d34e7e99abf6b6f367309975d1360222d446,0xf97d756f37c333657f425505a61d9e3c12329a6b69aa...,0xc4ad29ba4b3c580e6d59105fff484999997675ff,199997895019529000000
5,4,14649612,609,0x989aeb4d175e16225e39e87d0d97a3360524ad80,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x96e6c23311919875b58c5324c976838192772d8fc236...,0xc4ad29ba4b3c580e6d59105fff484999997675ff,-199997895019529000000
6,7,15672527,325,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x9e3feaf5f0483b2e196db31635734f627fdfd254,0x3dbaf50cdb30395c651500ef13592a109b8eca9f97fe...,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,100000
7,5,15672537,239,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x9e3feaf5f0483b2e196db31635734f627fdfd254,0x32f07bfee2a3382fe752e2599b8354627bbe9511ee52...,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,942140842
8,0,15676502,281,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x8ec4b391596ceed7e588cfd8bf7725d02aa42ab3a286...,0xeb4c2781e4eba804ce9a9803c67d0893436bb27d,-1000000
9,6,15676502,291,0x5291fbb0ee9f51225f0928ff6a83108c86327636,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0x8ec4b391596ceed7e588cfd8bf7725d02aa42ab3a286...,0xdac17f958d2ee523a2206206994597c13d831ec7,196895601


### The Tokens

In [18]:
wBTC = Token(address='0x2260fac5e5542a773aa44fbcfedf7c193bc2c599')
wBTC.scaled(748139023)

renBTC = Token(address='0xeb4c2781e4eba804ce9a9803c67d0893436bb27d')
renBTC

t1 = Token(address='0xdac17f958d2ee523a2206206994597c13d831ec7')
t2 = Token(address='0x2260fac5e5542a773aa44fbcfedf7c193bc2c599')
t3 = Token(address='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')

In [19]:
curve_usd_btc_eth = Token(address='0xc4ad29ba4b3c580e6d59105fff484999997675ff')
print(curve_usd_btc_eth.balance_of(acc_1.address))

ve_crv = Token(address='0x5f3b5DfEb7B28CDbD7FAba78963EE202a494e2A2')
print(ve_crv.balance_of(acc_1.address))
print(ve_crv.functions.locked(acc_1.address).call())

convex_voter_proxy = Contract(address='0x989AEb4d175e16225E39E87d0D97A3360524AD80')
convex_voter_proxy.abi.functions

0
0
[0, 0]


Functions: [balanceOfPool, claimCrv, claimFees, claimRewards, createLock, crv, deposit, depositor, escrow, execute, gaugeController, getName, increaseAmount, increaseTime, mintr, operator, owner, release, setDepositor, setOperator, setOwner, setStashAccess, vote, voteGaugeWeight, withdraw, withdrawAll]

In [20]:
from decimal import Decimal
pos_dict = {}
for n, r in df_tt.iterrows():
    tok_address = r['token_address']
    value = r['sum_value']
    pos_dict[tok_address] = pos_dict.get(tok_address, Decimal(0)) + Decimal(value)
    if pos_dict[tok_address] == 0:
        tok = Token(address=tok_address)
        print(tok.symbol, tok_address)
        del pos_dict[tok_address]

display(pos_dict)

token_dict = {}
for tok_address, amount in pos_dict.items():
    tok = Token(address=tok_address)
    token_dict[tok_address] = tok
    pos_dict[tok_address] = tok.scaled(amount)

pos_dict

WBTC 0x2260fac5e5542a773aa44fbcfedf7c193bc2c599
crv3crypto 0xc4ad29ba4b3c580e6d59105fff484999997675ff
renBTC 0xeb4c2781e4eba804ce9a9803c67d0893436bb27d
crv3crypto 0xc4ad29ba4b3c580e6d59105fff484999997675ff
crv3crypto 0xc4ad29ba4b3c580e6d59105fff484999997675ff
crv3crypto 0xc4ad29ba4b3c580e6d59105fff484999997675ff
CRV 0xd533a949740bb3306d119cc777fa900ba034cd52
CVX 0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b
crv3crypto 0xc4ad29ba4b3c580e6d59105fff484999997675ff


{'0xdac17f958d2ee523a2206206994597c13d831ec7': Decimal('-6356071645'),
 '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': Decimal('2169356')}

{'0xdac17f958d2ee523a2206206994597c13d831ec7': Decimal('-6356.071645'),
 '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': Decimal('2.169356')}

### Curve Pool

In [21]:
pool_info = context.models.curve_fi.pool_info(address='0xc4ad29ba4b3c580e6d59105fff484999997675ff')
pool_info = context.models.curve_fi.pool_info(address=pool_info['address'])
pool_info

{'address': '0xd51a44d3fae010294c616388b506acda1bfaae46',
 'tokens': {'tokens': [{'address': '0xdac17f958d2ee523a2206206994597c13d831ec7'},
   {'address': '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599'},
   {'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'}]},
 'tokens_symbol': ['USDT', 'WBTC', 'WETH'],
 'balances': [74717825.210047, 2500.92622388, 35709.63153108596],
 'balances_token': [74717825.210047, 2500.92622388, 35709.63153108596],
 'admin_fees': [0, 0, 0],
 'underlying': {'tokens': []},
 'underlying_symbol': [],
 'name': '',
 'lp_token_name': 'Curve.fi USD-BTC-ETH',
 'lp_token_addr': '0xc4ad29ba4b3c580e6d59105fff484999997675ff',
 'token_prices': [{'price': 1.001,
   'src': 'cex|chainlink.price-by-registry|USDT / USD|0xa964273552C1dBa201f5f000215F5BD5576e8f93|v4|True|t:52560s|r:0',
   'quoteAddress': '0x0000000000000000000000000000000000000348'},
  {'price': 30005.27,
   'src': 'cex|chainlink.price-by-registry|BTC / USD|0xAe74faA92cB67A95ebCAB07358bC222e33A34dA7|v4|True|t:2

In [22]:
pool = Contract(address='0xd51a44d3fae010294c616388b506acda1bfaae46')
b1,b2,b3=(pool.functions.balances(0).call(),
          pool.functions.balances(1).call(),
          pool.functions.balances(2).call())
pool

Contract(address='0xd51a44d3fae010294c616388b506acda1bfaae46')

In [23]:
cc = Contract(address=pool_info['gauges']['accounts'][0]['address'])
print(cc.functions.balanceOf(acc_1.address.checksum).call())
print(cc.functions.working_balances(acc_1.address.checksum).call())
print(cc.functions.claimable_tokens(acc_1.address.checksum).call())

0
0
0
